In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
from collections import defaultdict
import math
from scipy import stats

In [2]:
from lib.moments import init_moment_k, cent_moment_k
from lib.sample_creation import get_sample

# **ЛР 3.1**

In [3]:
ages = []
with open('moscow_2021.txt', 'r') as f:
    for l in f.readlines():
        ages.append(int(l.strip()))

### _Функции моментов для сгруппированного ряда_

In [5]:
def init_moment_k_interval(val_array: list, freq_array: list, k=1):
    total_vf=0
    freq_sum = sum(freq_array)
    for val, freq in zip(val_array, freq_array):
        total_vf += freq*(val)**k
    return total_vf/freq_sum


def cent_moment_k_interval(val_array: list, freq_array: list, k):
    mean_val = init_moment_k_interval(val_array, freq_array, 1)

    total_vf=0
    freq_sum = sum(freq_array)
    for val, freq in zip(val_array, freq_array):
        total_vf += freq*(val-mean_val)**k
    return total_vf/freq_sum

### _Функции для создания интервальных рядов_

In [7]:
def make_interval_freq_list(array: list, delta_x: int) -> list:
    '''inter_array[0] = count[min_val; min_val + delta_x)

    inter_array[1] = count[min + delta_x; min + 2*delta_x)

    ...'''    
    nice_array = sorted(array)
    if len(nice_array)==0:
        return []
    
    min_val = math.floor(nice_array[0])
    max_val = math.ceil(nice_array[-1])
    num_intervals = math.ceil((max_val-min_val)/delta_x)

    interval_array = [0]*num_intervals
    for a in array:
        interval_array[math.floor(a-min_val)//delta_x]+=1
    return interval_array


def make_interval_median_list(min_val: float, max_val: float, delta_x: int) -> list:
    num_intervals = math.ceil((max_val-min_val)/delta_x)
    interval_median_array = [0.]*num_intervals
    
    curr_val = min_val
    for i in range(num_intervals):
        interval_median_array[i] = (2*curr_val+delta_x)/2
        curr_val += delta_x

    return interval_median_array

### _Функции для нормирования интервального ряда_

In [8]:
def get_interval_borders(min_val, max_val, delta_x) -> list:
    num_intervals = math.ceil((max_val-min_val)/delta_x)
    return [min_val+i*delta_x for i in range(num_intervals+1)]


def norm_vals(val_list: list, mean_val: float, stand_dev: float) -> list:
    val_array = np.array(val_list)
    val_array = (val_array - mean_val)/stand_dev
    return val_array.tolist()
    

### _Функции для критерия согласия Пирсона_

$\chi^2_{obs} = \sum\limits_{i=1}^{s}\frac{(n_i-n_i\rq)^2}{n_i\rq} = (\sum\limits_{i=1}^{s}\frac{n_i^2}{n_i\rq}) - n$

In [9]:
def make_theor_freq_interval_list(array_interval_borders: list,
                                   mean_val: float,
                                   stand_dev: float,
                                   total_freq: int) -> list:
    laplace = lambda x: stats.norm.cdf(x)-0.5
    # total_freq = sum(array_interval_freq)

    array_interval_borders_normed = np.array(norm_vals(array_interval_borders, mean_val, stand_dev))
    
    array_interval_borders_normed_laplaced = laplace(array_interval_borders_normed)
    array_interval_borders_normed_laplaced[0] = -0.5 # we think that left border is negative infinity
    array_interval_borders_normed_laplaced[-1] = 0.5 # we think that right border is positive infinity 

    num_intervals = len(array_interval_borders_normed_laplaced)-1
    array_interval_norm_probability = [0]*num_intervals
    for i in range(num_intervals):
        array_interval_norm_probability[i] = array_interval_borders_normed_laplaced[i+1]-array_interval_borders_normed_laplaced[i]

    return (np.array(array_interval_norm_probability)*total_freq).tolist()


def chi2_observed(freq_list: list, theory_freq_list: list) -> float:
    freq_array = np.array(freq_list)
    theory_freq_array = np.array(theory_freq_list)
    total_freq = sum(freq_array)
    
    division_array = (freq_array**2)/theory_freq_array
    return sum(division_array)-total_freq

def chi2_crit(alpha: float, df: int) -> float:
    return stats.chi2.ppf(1-alpha, df)

## **Ход работы**

### ***Условия***

In [10]:
ages_sample_size = 62
ages_samples_number = 36
age_delta = 9

age_alpha = 0.05

### ***Вычисления***

#### _Интервальный ряд распределения возраста людей_

In [11]:
min_age = min(ages)
max_age = max(ages)

In [12]:
ages_interval_freq = make_interval_freq_list(ages, age_delta)
ages_interval_medians = make_interval_median_list(min_age, max_age, age_delta)

ages_interval_mean = init_moment_k_interval(ages_interval_medians, ages_interval_freq)
ages_interval_stand_dev = cent_moment_k_interval(ages_interval_medians, ages_interval_freq, 2)**0.5
ages_total_freq = sum(ages_interval_freq)

In [13]:
ages_interval_borders = get_interval_borders(min_age, max_age, age_delta)
ages_interval_theory_freq = make_theor_freq_interval_list(ages_interval_borders, ages_interval_mean, ages_interval_stand_dev, ages_total_freq)

In [14]:
ages_interval_chi2_observ = chi2_observed(ages_interval_freq, ages_interval_theory_freq)
ages_interval_chi2_crit = chi2_crit(age_alpha, ages_total_freq-1)

print(f'Pirson criteria for ages_interval:')
print(f'chi2_observed = {ages_interval_chi2_observ:.2f}')
print(f'chi2_critical = {ages_interval_chi2_crit:.2f}')

cmpr_res = '>' if ages_interval_chi2_observ>ages_interval_chi2_crit else '<='
print(f'chi2_observed {cmpr_res} chi2_critical')

Pirson criteria for ages_interval:
chi2_observed = 3233.27
chi2_critical = 32841.99
chi2_observed <= chi2_critical


#### _Интервальный ряд распределения среднего возраста людей_

In [15]:
ages_sample_means = []
for _ in range(ages_samples_number):
    ages_sample_means.append(init_moment_k(get_sample(ages, ages_sample_size), 1))


min_ages_sample_means = math.floor(min(ages_sample_means))
max_ages_sample_means = math.ceil(max(ages_sample_means))
delta_ages_sample_means = 1

In [16]:
ages_sample_means_interval_freq = make_interval_freq_list(ages_sample_means,
                                                          delta_ages_sample_means)
ages_sample_means_interval_medians = make_interval_median_list(min_ages_sample_means,
                                                               max_ages_sample_means,
                                                               delta_ages_sample_means)


ages_sample_means_interval_mean = init_moment_k_interval(ages_sample_means_interval_medians,
                                                         ages_sample_means_interval_freq)
ages_sample_means_interval_stand_dev = cent_moment_k_interval(ages_sample_means_interval_medians,
                                                              ages_sample_means_interval_freq, 2)**0.5
ages_sample_means_total_freq = sum(ages_sample_means_interval_freq)


ages_sample_means_interval_borders = get_interval_borders(min_ages_sample_means,
                                                          max_ages_sample_means,
                                                          delta_ages_sample_means)
ages_sample_means_interval_theory_freq = make_theor_freq_interval_list(ages_sample_means_interval_borders,
                                                                       ages_sample_means_interval_mean,
                                                                       ages_sample_means_interval_stand_dev,
                                                                       ages_sample_means_total_freq)

In [17]:
ages_sample_means_interval_chi2_observ = chi2_observed(ages_sample_means_interval_freq,
                                                       ages_sample_means_interval_theory_freq)
ages_sample_means_interval_chi2_crit = chi2_crit(age_alpha,
                                                 ages_sample_means_total_freq-1)

print(f'Pirson criteria for ages_interval:')
print(f'chi2_observed = {ages_sample_means_interval_chi2_observ:.2f}')
print(f'chi2_critical = {ages_sample_means_interval_chi2_crit:.2f}')

cmpr_res = '>' if ages_sample_means_interval_chi2_observ>ages_sample_means_interval_chi2_crit else '<='
print(f'chi2_observed {cmpr_res} chi2_critical')

Pirson criteria for ages_interval:
chi2_observed = 6.05
chi2_critical = 49.80
chi2_observed <= chi2_critical
